In [7]:
import requests
from datetime import datetime
from kaggle_secrets import UserSecretsClient
import pandas as pd
import time

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("FPM_KEY")

# On utilise la fonction écrite par Samuel
def get_historical_price(symbol, from_date, to_date, api_key):
    """
    Récupère les données historiques des prix pour un symbole donné.

    Args:
        symbol (str): Le symbole de l'action ou ETF
        from_date (str): Date de début au format YYYY-MM-DD
        to_date (str): Date de fin au format YYYY-MM-DD
        api_key (str): Clé API pour Financial Modeling Prep

    Returns:
        pandas.DataFrame: DataFrame contenant les données historiques
    """
    base_url = "https://financialmodelingprep.com/api/v3/historical-price-full/"
    url = f"{base_url}{symbol}?from={from_date}&to={to_date}&apikey={api_key}"

    response = requests.get(url)

    if response.status_code == 200: # Si la requête est OK (succès)
        data = response.json()
        if "historical" in data:
            historical_data = data["historical"]
            df = pd.DataFrame(historical_data)
            # Convertir la colonne 'date' en datetime et la définir comme index
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values('date')  # Trier par date croissante
            return df
        else:
            print(f"Pas de données historiques pour {symbol}")
            return None
    else:
        print(f"Erreur de requête pour {symbol}: {response.status_code}")
        return None



In [10]:
# Récupération des tickers NASDAQ sur wikipedia
url = 'https://en.wikipedia.org/wiki/NASDAQ-100'
tables = pd.read_html(url)
nasdaq100 = tables[4]  
tickers = nasdaq100['Ticker'].tolist()

# Paramètres pour la requête
from_date = "2020-01-01"
to_date = "2024-12-31"

# On crée un df vide price 
df_price = pd.DataFrame()

# Récupérer les données pour chaque ticker
for ticker in tickers:
    print(f"\nRécupération des données pour {ticker}...")
    df = get_historical_price(ticker, from_date, to_date, api_key)
    df['symbol'] = ticker

    if df is not None:
        # On merge ici chaque requête dans le df vide
        df_price = pd.concat([df_price, pd.DataFrame(df)], axis = 0)
    else:
        print(f"✗ Échec de récupération des données pour {ticker}")

    # Ajouter un délai pour éviter de surcharger l'API
    time.sleep(1)

print("\nTraitement terminé pour tous les tickers.")


Récupération des données pour ADBE...

Récupération des données pour AMD...

Récupération des données pour ABNB...

Récupération des données pour GOOGL...

Récupération des données pour GOOG...

Récupération des données pour AMZN...

Récupération des données pour AEP...

Récupération des données pour AMGN...

Récupération des données pour ADI...

Récupération des données pour ANSS...

Récupération des données pour AAPL...

Récupération des données pour AMAT...

Récupération des données pour APP...

Récupération des données pour ARM...

Récupération des données pour ASML...

Récupération des données pour AZN...

Récupération des données pour TEAM...

Récupération des données pour ADSK...

Récupération des données pour ADP...

Récupération des données pour AXON...

Récupération des données pour BKR...

Récupération des données pour BIIB...

Récupération des données pour BKNG...

Récupération des données pour AVGO...

Récupération des données pour CDNS...

Récupération des données pour C

In [17]:
display(df_price.head())
df_price.to_csv('/kaggle/working/historical_ohlc_nasdaq100.csv', index=False)

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
1257,2020-01-02,330.00,334.48,329.170,334.43,334.429993,1990496,1990100,4.43,1.34000,332.94,"January 02, 20",0.013400,ADBE
1256,2020-01-03,329.17,332.98,328.690,331.81,331.809998,1579371,1577600,2.64,0.80202,331.84,"January 03, 20",0.008020,ADBE
1255,2020-01-06,328.29,333.91,328.200,333.71,333.709991,1875122,1874700,5.42,1.65000,332.42,"January 06, 20",0.016500,ADBE
1254,2020-01-07,334.15,334.79,332.305,333.39,333.390000,2507261,2500800,-0.76,-0.22744,333.80,"January 07, 20",-0.002274,ADBE
1253,2020-01-08,333.81,339.23,333.410,337.87,337.869995,2248531,2248500,4.06,1.22000,337.20,"January 08, 20",0.012200,ADBE
